In [1]:
import re
from collections import defaultdict
import pickle
from random import choice

In [2]:
def gen_lines_from_files(list_of_corpuses):
    for corpus in list_of_corpuses:  
        with open(corpus, 'r', encoding='utf-8') as file:
            for line in file:
                yield line.lower()
                

def gen_tokens(lines):
    alphabet = re.compile(u'[а-яА-Я0-9]+')
    for line in lines:
        for token in alphabet.findall(line):
            yield token

            
def gen_trigrams(tokens):
    t0, t1 = next(tokens), next(tokens)
    for t2 in tokens:
        yield t0, t1, t2
        t0, t1 = t1, t2

        
def fit(list_of_corpuses):
    lines = gen_lines_from_files(list_of_corpuses)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)
    bi, tri = defaultdict(int) , defaultdict(int) 
    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1
    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]  
    return model


def generate(model, length, prefix='-1'):
    phrase = ''
    bigrams = tuple(filter(lambda key: key[0] == prefix, model))
    #Если ввели 2 слова
    if tuple(prefix.split()) in model:
        t0, t1 = prefix.split()      
    # Если ввели одно слово и оно есть в биграммах
    elif prefix != '-1' and len(bigrams) != 0:
        t0, t1 = choice(tuple(filter(lambda key: key[0] == prefix, model)))
    # Введеного слова нет в биграммах или ничего не ввели
    else:
        t0, t1 = choice(tuple(model))
    phrase += t0
    for i in range(length-len(prefix.split())):
        phrase += ' ' + t1
        t0, t1 = t1, choice(model[t0, t1])[0]
    return phrase.capitalize() + '.'


In [3]:
model = fit(['data/Disgardium_5.txt', 'data/tolstoy.txt'])
print(generate(model=model, length=50, prefix='закричал он'))

Закричал он подпрыгивая по льду который трещал под ним сказал папа прошу любить и своего отечества к общему кружку анна павловна отвернулись от меня нам вообще биться нельзя прогудел бабангида поддерживаю огрилу встрял патрик кровожадно сжимающий в руках старое бальное платье и каких то сагрившихся мобов в кадре появился.


In [4]:
model

{('дисгардиум', '5'): [('священная', 0.6666666666666666),
  ('под', 0.3333333333333333)],
 ('5', 'священная'): [('война', 1.0)],
 ('священная', 'война'): [('данияр', 0.2),
  ('краткое', 0.2),
  ('официально', 0.2),
  ('начата', 0.2),
  ('закончена', 0.2)],
 ('война', 'данияр'): [('саматович', 1.0)],
 ('данияр', 'саматович'): [('сугралинов', 1.0)],
 ('саматович', 'сугралинов'): [('дисгардиум', 1.0)],
 ('сугралинов', 'дисгардиум'): [('5', 1.0)],
 ('5', 'под'): [('защитой', 1.0)],
 ('под', 'защитой'): [('нергала', 0.3333333333333333),
  ('краша', 0.3333333333333333),
  ('слабых', 0.3333333333333333)],
 ('защитой', 'нергала'): [('лучезарного', 1.0)],
 ('нергала', 'лучезарного'): [('армии', 0.14285714285714285),
  ('личность', 0.14285714285714285),
  ('который', 0.14285714285714285),
  ('во', 0.14285714285714285),
  ('уже', 0.14285714285714285),
  ('воздух', 0.14285714285714285),
  ('сплюнув', 0.14285714285714285)],
 ('лучезарного', 'армии'): [('союзников', 1.0)],
 ('армии', 'союзников'): [

In [5]:
print(generate(model=model, length=50, prefix='ученые'))

Ученые хоть из примеров красного и поехал за ними солдаты и офицеры несли большую с черным ликом в окладе икону это была последняя надежда и исполненное интереса наблюдение за деятельностью своего тела то перед глазами всегда было готово сабли воткнуты в снег означая барьер до которого был приятель доктор он.


In [6]:
print(generate(model=model, length=30))

Из 50000 людей а таких людей нынче уж не в сложении но в глубине же души аксинья была рада узнав о вашем поединке с угрозой закончилась твоя слава старик промелькнуло.


In [7]:
# Запись в файл
filename = 'trigram_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [8]:
# Загрузка модели из файла
filename = 'trigram_model.pickle'
with open(filename, 'rb') as file:
    model_from_pickle = pickle.load(file)


In [9]:
model_from_pickle

{('дисгардиум', '5'): [('священная', 0.6666666666666666),
  ('под', 0.3333333333333333)],
 ('5', 'священная'): [('война', 1.0)],
 ('священная', 'война'): [('данияр', 0.2),
  ('краткое', 0.2),
  ('официально', 0.2),
  ('начата', 0.2),
  ('закончена', 0.2)],
 ('война', 'данияр'): [('саматович', 1.0)],
 ('данияр', 'саматович'): [('сугралинов', 1.0)],
 ('саматович', 'сугралинов'): [('дисгардиум', 1.0)],
 ('сугралинов', 'дисгардиум'): [('5', 1.0)],
 ('5', 'под'): [('защитой', 1.0)],
 ('под', 'защитой'): [('нергала', 0.3333333333333333),
  ('краша', 0.3333333333333333),
  ('слабых', 0.3333333333333333)],
 ('защитой', 'нергала'): [('лучезарного', 1.0)],
 ('нергала', 'лучезарного'): [('армии', 0.14285714285714285),
  ('личность', 0.14285714285714285),
  ('который', 0.14285714285714285),
  ('во', 0.14285714285714285),
  ('уже', 0.14285714285714285),
  ('воздух', 0.14285714285714285),
  ('сплюнув', 0.14285714285714285)],
 ('лучезарного', 'армии'): [('союзников', 1.0)],
 ('армии', 'союзников'): [

In [10]:
print(generate(model=model_from_pickle, length=30, prefix='эд'))

Эд ничего не думал о вчерашнем бале и хотя и слабо шагая плохо повинующимися старческими ногами стал переходить из гостиной выбежал петя петя был уже произведен в офицеры когда она.


In [11]:
list_of_corpuses = ['C:\\Users\\home\\EXAM_TINKOFF\\new-project\\data\\Disgardium_5.txt', 
                    'C:\\Users\\home\\EXAM_TINKOFF\\new-project\\data\\tolstoy.txt']

In [15]:
# С помощью класса

class MyTrigramModel:
    
    def __init__(self):
        pass
    
    @staticmethod
    def fit(list_of_corpuses):
        lines = gen_lines_from_files(list_of_corpuses)
        tokens = gen_tokens(lines)
        trigrams = gen_trigrams(tokens)

        bi, tri = defaultdict(int), defaultdict(int)

        for t0, t1, t2 in trigrams:
            bi[t0, t1] += 1
            tri[t0, t1, t2] += 1

        model = {}
        for (t0, t1, t2), freq in tri.items():
            if (t0, t1) in model:
                model[t0, t1].append((t2, freq / bi[t0, t1]))
            else:
                model[t0, t1] = [(t2, freq / bi[t0, t1])]
        return model

    @staticmethod
    def generate(model, length, prefix='-1'):
        phrase = ''
        bigrams = tuple(filter(lambda key: key[0] == prefix, model))
        # Если ввели 2 слова
        if tuple(prefix.split()) in model:
            t0, t1 = prefix.split()

        # Если ввели одно слово и оно есть в биграммах
        elif prefix != '-1' and len(bigrams) != 0:
            t0, t1 = choice(tuple(filter(lambda key: key[0] == prefix, model)))

        # Введеного слова нет в биграммах или ничего не ввели
        else:
            t0, t1 = choice(tuple(model))
        phrase += t0
        for i in range(length - len(prefix.split())):
            phrase += ' ' + t1
            t0, t1 = t1, choice(model[t0, t1])[0]
        return phrase.capitalize() + '.'

In [16]:
my_m = MyTrigramModel.fit(list_of_corpuses)

In [22]:
print(MyTrigramModel.generate(my_m, 30, 'я знаю'))

Я знаю чьи это интриги я знаю у вас снова жар вы хотите сжечь дом погубить всех нас значит всех товарищей более или менее развитая в различных кружках общества.
